In [1]:
import os 
%pwd

's:\\study\\data scientist\\projects\\End-To-End MLOps\\MLOps\\research'

In [2]:
os.chdir("../")
%pwd

's:\\study\\data scientist\\projects\\End-To-End MLOps\\MLOps'

In [51]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [52]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories


In [53]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, 
                 params_filepath=PARAMS_FILE_PATH, 
                 scheme_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(str(config_filepath))
        self.params = read_yaml(str(params_filepath))
        self.scheme = read_yaml(str(scheme_filepath))


        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> DataIngestionConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        scheme = self.scheme.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = DataIngestionConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=scheme.name,
            mlflow_uri="https://dagshub.com/preetdhanani/MLOps.mlflow"
        )
        return model_evaluation_config

In [54]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
from mlProject.utils.common import save_json
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
import dagshub
dagshub.init(repo_owner='preetdhanani', repo_name='MLOps', mlflow=True)

[2025-06-08 12:34:27,499: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/preetdhanani/MLOps "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "preetdhanani/MLOps"

[2025-06-08 12:34:27,509: INFO: helpers: Initialized MLflow to track repo "preetdhanani/MLOps"]


Repository preetdhanani/MLOps initialized!

[2025-06-08 12:34:27,511: INFO: helpers: Repository preetdhanani/MLOps initialized!]


In [ ]:
class ModelEvaluation:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    

    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
    
        X_test = test_data.drop([self.config.target_column], axis=1)
        y_test = test_data[[self.config.target_column]]
        
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_qualities = model.predict(X_test)

            (rmse, mae, r2) = self.eval_metrics(y_test, predicted_qualities)

            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=self.config.metric_file_name, data=scores)
            
            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)

            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")
    


In [56]:
try:
    config = ConfigurationManager().get_model_evaluation_config()
    model_eval = ModelEvaluation(config)
    model_eval.log_into_mlflow()
except Exception as e:      
    
    raise e

[2025-06-08 12:34:27,554: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-08 12:34:27,564: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-08 12:34:27,566: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-06-08 12:34:27,567: INFO: common: Created directory at: artifacts]
[2025-06-08 12:34:27,568: INFO: common: Created directory at: artifacts/model_evaluation]
[2025-06-08 12:34:27,817: INFO: common: JSON file saved at: artifacts/model_evaluation/metrics.json]


2025/06/08 12:34:31 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2025/06/08 12:34:34 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetModel, version 4
Created version '4' of model 'ElasticnetModel'.


🏃 View run capable-shrike-7 at: https://dagshub.com/preetdhanani/MLOps.mlflow/#/experiments/0/runs/424f6b1f95734d13aec08f52667241b6
🧪 View experiment at: https://dagshub.com/preetdhanani/MLOps.mlflow/#/experiments/0
